This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the complex model with interaction terms that was run on the cell-level data (not pseudo-bulk, no holdout analysis). This script contains the code that was run on the full data and does not test for smoking associations.

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_nUMIoffset_testInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

In [10]:
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()
adata.obs['dataset'].nunique()

377

228

31

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [11]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [12]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [13]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Schultze/Falk', 'Linnarsson', 'Rawlins', 'Misharin', 'Spence']

In [14]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [15]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [16]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [17]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [18]:
# Rename smoking status covariate
adata.obs['smoking_status'] = adata.obs.smoked_boolean

In [19]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Xavier/Regev                25552
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

309

185

# Check the data

In [20]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [21]:
# Check if any non-integer data in a particular dataset
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are integers, or are very close to integers!

In [22]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
55.50    27176
20.00    25362
51.00    24766
72.50    24366
30.00    24095
58.00    23723
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
45.50    16351
56.00    16011
23.00    15181
41.00    14943
49.00    14225
57.50    13840
55.00    13733
68.00    11852
65.00    11404
75.00    11243
45.00    10939
47.00    10278
52.50    10133
21.00     9430
67.00     9278
27.00     8693
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
62.00     6228
38.00     5629
73.00     5251
24.00     4433
60.00     4231
62.50     4073
26.00     4066
61.00     3922
50.00     3831
32.50     3772
52.00     3765
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
53.00     1592
37.00     1240
48.00     1153
81.00     1145
25.00      823
70.00     

female    583926
male      512678
Name: sex, dtype: int64

# Fit models and perform DE

In [23]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium',
 'Endothelial-like',
 'Granulocytes']

Calculate DE genes per cluster.

In [24]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

In [25]:
adata.obs['total_counts_scaled'] = adata.obs['total_counts']/adata.obs['total_counts'].mean()

In [26]:
# Get interquartile range for ages to test
adata.obs.groupby(['donor']).agg({'age':'first'}).age.quantile([0.25,0.5,0.75])

0.25    32.0
0.50    52.0
0.75    62.5
Name: age, dtype: float64

In [27]:
formula = "1 + sex + age + sex:age + dataset"
tested_coef = ["sex[T.male]", "age"]
dmat = de.utils.design_matrix(
    data=adata,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)

to_test = dict()
to_test['age'] = [32,62]
to_test['sex[T.male]'] = [0,1]

dmat[1]

['Intercept',
 'sex[T.male]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Schultze]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'dataset[T.Xavier/Regev]',
 'age',
 'sex[T.male]:age']

### Function definition to test effect sizes at particular covariate values

In [28]:
def calc_effects(dmat, cov_mat, params, effect, coefs):
    from patsy.design_info import DesignMatrix
    from diffxpy.api.stats import wald_test_chisq
    
    dmat_cond = isinstance(dmat, tuple) and isinstance(dmat[0], DesignMatrix)

    if not dmat_cond:
        raise ValueError("`dmat` should be a patsy output Design Matrix.")

    effect_list = ['sex[T.male]', 'age', 'smoking_status[T.True]']
    if not effect in effect_list:
        raise ValueError(f'{effect} is not one of: '
                         f'{effect_list}')

    if not isinstance(coefs, dict):
        raise TypeError('`coefs` should contain a dictionary of coefficients '
                        'where the effects should be evaluated.')
    
    ## Note: this is only correct when 3 covariates are tested in combinations
    #if np.sum([coef in coefs for coef in effect_list]) < 2:
    #    raise ValueError('The `coefs` dict must contain values for the two '
    #                     'coefficient not tested in:'
    #                     f'{effect_list}')
        
    if 'smoking_status[T.True]' in coefs and coefs['smoking_status[T.True]'] not in [0,1]:
        raise ValueError('Smoking status should be encoded as 0 or 1.')

    if 'sex[T.male]' in coefs and coefs['sex[T.male]'] not in [0,1]:
        raise ValueError('Sex should be encoded as 0 or 1.')

    if 'age' in coefs and not (isinstance(coefs['age'], float) or isinstance(coefs['age'], int)):
        raise ValueError('Age should be a numerical value.')

    coef_list = []

    for term in dmat[1]:
        if effect not in term:
            coef_list.append(0)
        elif term == effect:
            coef_list.append(1)
        else:
            t_list = term.split(':')
            t_list.remove(effect)
            coef_list.append(coefs[t_list[0]])

    C = np.array(coef_list)
    val = np.matmul(C,np.array(params))
    stderr = np.sqrt(np.matmul(np.matmul(C.T,cov_mat),C))
    pval = wald_test_chisq(np.array([val]).reshape(1,1), np.array([stderr**2]).reshape(1,1,1))[0]
    return (val, stderr, pval)

## Poisson GLM

In [29]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata[adata.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp,
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0], 
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                coef_vals = {iter_coefs[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                #for c2 in to_test[iter_coefs[1]]:
                #    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                #    val, stderr, pval = calc_effects(
                #        dmat = dmat,
                #        cov_mat = cov_mat,
                #        params = pois_results.params,
                #        effect = coef,
                #        coefs = coef_vals)
                    
                #case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                #case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                case = iter_coefs[0]+':'+str(c1)
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:32','32yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)
                    
    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
False    137441
True     109516
nan       36510
Name: smoking_status, dtype: int64
male      152540
female    130927
Name: sex, dtype: int64
Testing 3 genes...

In cluster Airway epithelium:
False    157085
True      61702
nan        8785
Name: smoking_status, dtype: int64
female    126940
male      100632
Name: sex, dtype: int64
Testing 3 genes...

In cluster Alveolar epithelium:
False    124700
True      60785
nan       36682
Name: smoking_status, dtype: int64
female    135518
male       86649
Name: sex, dtype: int64
Testing 3 genes...

In cluster Lymphoid:
False    84562
True     48215
nan       9664
Name: smoking_status, dtype: int64
female    85352
male      57089
Name: sex, dtype: int64
Testing 3 genes...

In cluster Fibroblast lineage:
False    40012
True     13154
nan       5455
Name: smoking_status, dtype: int64
female    32623
male      25998
Name: sex, dtype: int64
Testing 3 genes...

In cluster Blood vessels:
False    21455
True     21064
nan       9121


## Inspect some results

In [30]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium', 'Granulocytes'])

In [31]:
full_res_lvl2_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],32yr,0.994277,0.269131,2.204139e-04,3.778524e-04
1,ACE2,Myeloid,sex[T.male],62yr,0.153591,0.242150,5.258983e-01,5.737072e-01
2,ACE2,Myeloid,age,F,0.015873,0.008312,5.616867e-02,7.489156e-02
3,ACE2,Myeloid,age,M,-0.012150,0.008231,1.399257e-01,1.679108e-01
4,TMPRSS2,Myeloid,sex[T.male],32yr,0.626547,0.059655,0.000000e+00,0.000000e+00
5,TMPRSS2,Myeloid,sex[T.male],62yr,0.478947,0.058551,3.330669e-16,6.661338e-16
6,TMPRSS2,Myeloid,age,F,-0.000341,0.001830,8.522688e-01,8.522688e-01
7,TMPRSS2,Myeloid,age,M,-0.005261,0.002303,2.232904e-02,3.349357e-02
8,CTSL,Myeloid,sex[T.male],32yr,-0.161315,0.002873,0.000000e+00,0.000000e+00
9,CTSL,Myeloid,sex[T.male],62yr,-0.043937,0.002881,0.000000e+00,0.000000e+00


In [32]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],32yr,0.994277,0.269131,0.000220,0.000378
1,ACE2,Myeloid,sex[T.male],62yr,0.153591,0.242150,0.525898,0.573707
2,ACE2,Myeloid,age,F,0.015873,0.008312,0.056169,0.074892
3,ACE2,Myeloid,age,M,-0.012150,0.008231,0.139926,0.167911
12,ACE2,Airway epithelium,sex[T.male],32yr,0.100034,0.045824,0.029033,0.034840
13,ACE2,Airway epithelium,sex[T.male],62yr,0.026012,0.046047,0.572139,0.572139
14,ACE2,Airway epithelium,age,F,0.004849,0.001574,0.002060,0.002747
15,ACE2,Airway epithelium,age,M,0.002382,0.001308,0.068734,0.074983
24,ACE2,Alveolar epithelium,sex[T.male],32yr,0.888541,0.064718,0.000000,0.000000
25,ACE2,Alveolar epithelium,sex[T.male],62yr,0.719960,0.071947,0.000000,0.000000


In [33]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'ACE2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],32yr,0.994277,0.269131,0.000220,0.000378
12,ACE2,Airway epithelium,sex[T.male],32yr,0.100034,0.045824,0.029033,0.034840
14,ACE2,Airway epithelium,age,F,0.004849,0.001574,0.002060,0.002747
24,ACE2,Alveolar epithelium,sex[T.male],32yr,0.888541,0.064718,0.000000,0.000000
25,ACE2,Alveolar epithelium,sex[T.male],62yr,0.719960,0.071947,0.000000,0.000000
26,ACE2,Alveolar epithelium,age,F,0.015845,0.001631,0.000000,0.000000
27,ACE2,Alveolar epithelium,age,M,0.010225,0.002777,0.000231,0.000231
48,ACE2,Fibroblast lineage,sex[T.male],32yr,0.394325,0.184338,0.032424,0.043233
49,ACE2,Fibroblast lineage,sex[T.male],62yr,-0.462597,0.222845,0.037906,0.045175
50,ACE2,Fibroblast lineage,age,F,0.013793,0.006097,0.023681,0.035521


In [34]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'TMPRSS2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
4,TMPRSS2,Myeloid,sex[T.male],32yr,0.626547,0.059655,0.000000e+00,0.000000e+00
5,TMPRSS2,Myeloid,sex[T.male],62yr,0.478947,0.058551,3.330669e-16,6.661338e-16
7,TMPRSS2,Myeloid,age,M,-0.005261,0.002303,2.232904e-02,3.349357e-02
16,TMPRSS2,Airway epithelium,sex[T.male],32yr,0.251825,0.014075,0.000000e+00,0.000000e+00
17,TMPRSS2,Airway epithelium,sex[T.male],62yr,0.104907,0.015044,3.096967e-12,9.290901e-12
18,TMPRSS2,Airway epithelium,age,F,0.003398,0.000496,7.133849e-12,1.712124e-11
19,TMPRSS2,Airway epithelium,age,M,-0.001499,0.000394,1.396106e-04,2.094160e-04
28,TMPRSS2,Alveolar epithelium,sex[T.male],32yr,0.401614,0.009370,0.000000e+00,0.000000e+00
29,TMPRSS2,Alveolar epithelium,sex[T.male],62yr,-0.118468,0.008021,0.000000e+00,0.000000e+00
30,TMPRSS2,Alveolar epithelium,age,F,0.025354,0.000230,0.000000e+00,0.000000e+00


# Level 3 annotation

In [35]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'AT2',
 'Basal',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'AT1',
 'Submucosal Secretory',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Myofibroblasts',
 'MDC',
 'Airway smooth muscle']

In [36]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 895743 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'total_counts_scaled'

185

309

## Poisson GLM

In [37]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata_sub[adata_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.sex.value_counts())==1:
        print(f'{clust} only has 1 type of male/female sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                coef_vals = {iter_coefs[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                #for c2 in to_test[iter_coefs[1]]:
                #    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                #    val, stderr, pval = calc_effects(
                #        dmat = dmat,
                #        cov_mat = cov_mat,
                #        params = pois_results.params,
                #        effect = coef,
                #        coefs = coef_vals)
                    
                #case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                #case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                case = iter_coefs[0]+':'+str(c1)
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:32','32yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
False    84537
True     72427
nan      32007
Name: smoking_status, dtype: int64
male      113983
female     74988
Name: sex, dtype: int64
Testing 3 genes...

In cluster AT2:
False    100593
True      54919
nan       26612
Name: smoking_status, dtype: int64
female    112907
male       69217
Name: sex, dtype: int64
Testing 3 genes...

In cluster Basal:
False    113299
True      42578
nan         501
Name: smoking_status, dtype: int64
female    91574
male      64804
Name: sex, dtype: int64
Testing 3 genes...

In cluster T cell lineage:
True     26540
False    25599
nan       7702
Name: smoking_status, dtype: int64
male      30198
female    29643
Name: sex, dtype: int64
Testing 3 genes...

In cluster Monocytes:
True     23487
False    19216
nan        790
Name: smoking_status, dtype: int64
male      21798
female    21695
Name: sex, dtype: int64
Testing 3 genes...

In cluster Multiciliated lineage:
False    26913
True     10617
nan       4469
Name: smoking_status, dt

## Inspect some results

In [38]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'AT2', 'Basal', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'AT1', 'Submucosal Secretory', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Myofibroblasts', 'MDC', 'Airway smooth muscle'])

In [39]:
full_res_lvl3_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],32yr,0.930120,0.334093,5.368994e-03,1.073799e-02
1,ACE2,Macrophages,sex[T.male],62yr,-0.201575,0.272548,4.595457e-01,5.514548e-01
2,ACE2,Macrophages,age,F,0.028696,0.010247,5.103919e-03,1.073799e-02
3,ACE2,Macrophages,age,M,-0.009027,0.009343,3.339579e-01,4.452772e-01
4,TMPRSS2,Macrophages,sex[T.male],32yr,-0.101809,0.080804,2.076855e-01,3.560323e-01
5,TMPRSS2,Macrophages,sex[T.male],62yr,-0.087148,0.081792,2.866549e-01,4.299824e-01
6,TMPRSS2,Macrophages,age,F,-0.001393,0.002616,5.943762e-01,6.484104e-01
7,TMPRSS2,Macrophages,age,M,-0.000904,0.002926,7.572103e-01,7.572103e-01
8,CTSL,Macrophages,sex[T.male],32yr,-0.063322,0.003209,0.000000e+00,0.000000e+00
9,CTSL,Macrophages,sex[T.male],62yr,-0.218074,0.003061,0.000000e+00,0.000000e+00


In [40]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],32yr,0.930120,0.334093,5.368994e-03,1.073799e-02
1,ACE2,Macrophages,sex[T.male],62yr,-0.201575,0.272548,4.595457e-01,5.514548e-01
2,ACE2,Macrophages,age,F,0.028696,0.010247,5.103919e-03,1.073799e-02
3,ACE2,Macrophages,age,M,-0.009027,0.009343,3.339579e-01,4.452772e-01
12,ACE2,AT2,sex[T.male],32yr,0.953700,0.070098,0.000000e+00,0.000000e+00
13,ACE2,AT2,sex[T.male],62yr,0.769128,0.076183,0.000000e+00,0.000000e+00
14,ACE2,AT2,age,F,0.013895,0.001712,4.440892e-16,5.921189e-16
15,ACE2,AT2,age,M,0.007743,0.003061,1.142608e-02,1.246481e-02
24,ACE2,Basal,sex[T.male],32yr,-0.041766,0.058319,4.738943e-01,4.977279e-01
25,ACE2,Basal,sex[T.male],62yr,0.103370,0.061120,9.078568e-02,1.089428e-01


In [41]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'ACE2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],32yr,0.930120,0.334093,5.368994e-03,1.073799e-02
2,ACE2,Macrophages,age,F,0.028696,0.010247,5.103919e-03,1.073799e-02
12,ACE2,AT2,sex[T.male],32yr,0.953700,0.070098,0.000000e+00,0.000000e+00
13,ACE2,AT2,sex[T.male],62yr,0.769128,0.076183,0.000000e+00,0.000000e+00
14,ACE2,AT2,age,F,0.013895,0.001712,4.440892e-16,5.921189e-16
15,ACE2,AT2,age,M,0.007743,0.003061,1.142608e-02,1.246481e-02
27,ACE2,Basal,age,M,0.006140,0.001642,1.846115e-04,2.769172e-04
60,ACE2,Multiciliated lineage,sex[T.male],32yr,0.354480,0.103867,6.429474e-04,1.254986e-03
62,ACE2,Multiciliated lineage,age,F,0.011557,0.003422,7.320753e-04,1.254986e-03
72,ACE2,AT1,sex[T.male],32yr,0.513293,0.205806,1.262892e-02,1.894338e-02


In [42]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
4,TMPRSS2,Macrophages,sex[T.male],32yr,-0.101809,0.080804,2.076855e-01,3.560323e-01
5,TMPRSS2,Macrophages,sex[T.male],62yr,-0.087148,0.081792,2.866549e-01,4.299824e-01
6,TMPRSS2,Macrophages,age,F,-0.001393,0.002616,5.943762e-01,6.484104e-01
7,TMPRSS2,Macrophages,age,M,-0.000904,0.002926,7.572103e-01,7.572103e-01
16,TMPRSS2,AT2,sex[T.male],32yr,0.317262,0.010595,0.000000e+00,0.000000e+00
17,TMPRSS2,AT2,sex[T.male],62yr,-0.149234,0.008428,0.000000e+00,0.000000e+00
18,TMPRSS2,AT2,age,F,0.025567,0.000248,0.000000e+00,0.000000e+00
19,TMPRSS2,AT2,age,M,0.010017,0.000402,0.000000e+00,0.000000e+00
28,TMPRSS2,Basal,sex[T.male],32yr,0.448393,0.025010,0.000000e+00,0.000000e+00
29,TMPRSS2,Basal,sex[T.male],62yr,0.159454,0.025990,8.510677e-10,1.458973e-09


In [43]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'TMPRSS2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
16,TMPRSS2,AT2,sex[T.male],32yr,0.317262,0.010595,0.000000e+00,0.000000e+00
17,TMPRSS2,AT2,sex[T.male],62yr,-0.149234,0.008428,0.000000e+00,0.000000e+00
18,TMPRSS2,AT2,age,F,0.025567,0.000248,0.000000e+00,0.000000e+00
19,TMPRSS2,AT2,age,M,0.010017,0.000402,0.000000e+00,0.000000e+00
28,TMPRSS2,Basal,sex[T.male],32yr,0.448393,0.025010,0.000000e+00,0.000000e+00
29,TMPRSS2,Basal,sex[T.male],62yr,0.159454,0.025990,8.510677e-10,1.458973e-09
30,TMPRSS2,Basal,age,F,0.010916,0.000858,0.000000e+00,0.000000e+00
31,TMPRSS2,Basal,age,M,0.001284,0.000590,2.949630e-02,3.932840e-02
40,TMPRSS2,T cell lineage,sex[T.male],32yr,0.841926,0.189414,8.793212e-06,2.110371e-05
43,TMPRSS2,T cell lineage,age,M,-0.023000,0.008670,7.982014e-03,1.596403e-02


In [44]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'CTSL',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
8,CTSL,Macrophages,sex[T.male],32yr,-0.063322,0.003209,0.000000e+00,0.000000e+00
9,CTSL,Macrophages,sex[T.male],62yr,-0.218074,0.003061,0.000000e+00,0.000000e+00
10,CTSL,Macrophages,age,F,0.005884,0.000109,0.000000e+00,0.000000e+00
11,CTSL,Macrophages,age,M,0.000726,0.000104,3.448908e-12,1.034672e-11
20,CTSL,AT2,sex[T.male],32yr,-0.026421,0.018651,1.565904e-01,1.565904e-01
21,CTSL,AT2,sex[T.male],62yr,0.201675,0.017688,0.000000e+00,0.000000e+00
22,CTSL,AT2,age,F,-0.004349,0.000363,0.000000e+00,0.000000e+00
23,CTSL,AT2,age,M,0.003254,0.000794,4.149886e-05,4.979863e-05
32,CTSL,Basal,sex[T.male],32yr,-0.217960,0.012758,0.000000e+00,0.000000e+00
33,CTSL,Basal,sex[T.male],62yr,-0.130591,0.013829,0.000000e+00,0.000000e+00


# Store results

In [51]:
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [52]:
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')